In [1]:
import os, sys, time

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable, Function

import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import pylab
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import loadseg
import expdir
import intersect
import upsample

In [3]:
directory='/home/ruthfong/NetDissect/probes/pytorch_alexnet_imagenet'
blob='features'
batch_size=16
tally_depth=2048
quantile=0.005
verbose=True

In [4]:
qcode = ('%f' % quantile).replace('0.','').rstrip('0')
print qcode

005


In [5]:
ed = expdir.ExperimentDirectory(directory)

In [6]:
info = ed.load_info()
ih, iw = info.input_dim
print ih, iw

227 227


In [7]:
blob_info = ed.load_info(blob=blob)
shape = blob_info.shape

In [10]:
shape

(63305, 256, 6, 6)

In [15]:
imgmax = ed.open_mmap(blob=blob, part='imgmax', shape=shape[:2])

In [17]:
tally = ed.open_mmap(blob=blob, part='tally-005', shape=(ds.size(), tally_depth, 3))

In [22]:
info.input_dim

[227, 227]

In [11]:
quantdata = ed.open_mmap(blob=blob,part='quant-*',shape=(shape[1],-1))
threshold = quantdata[:, int(round(quantdata.shape[1]*quantile))]
thresh = threshold[:, np.newaxis, np.newaxis]

In [12]:
fn_read = ed.mmap_filename(blob=blob)

In [13]:
ds = loadseg.SegmentationData(info.dataset)
categories = ds.category_names()

In [23]:
ds.label_size()

1198

In [190]:
from labelprobe import onehot, primary_categories_per_index, cached_memmap

In [191]:
labelcat = onehot(primary_categories_per_index(ds, categories))

In [192]:
label_i = 237

In [204]:
print [cat for cat in catmap if label_i in catmap[cat]]

[u'object', u'scene']


In [203]:
np.sum(image_to_label[:,label_i])

0

In [205]:
categories = ds.label[label_i]['category'].keys()
print categories

[u'texture']


In [172]:
x = True
print x == []

False


In [197]:
fn_t = ed.mmap_filename(blob=blob, part='tally-%s' % qcode, inc=False)
print fn_t

/home/ruthfong/NetDissect/probes/pytorch_alexnet_imagenet/features-tally-005.mmap


In [198]:
blobdata = cached_memmap(fn_read, mode='r', dtype='float32', shape=shape)

In [199]:
from indexdata import load_image_to_label

In [200]:
image_to_label = load_image_to_label(directory)

In [201]:
#label_i = 19 # person

In [202]:
idx = np.where(image_to_label[:,label_i])[0]
print len(idx)

0


In [40]:
sys.path.append('../../net_concepts/src')

In [41]:
from learn_concept_masks import *

In [42]:
label_cat = categories[np.where(labelcat[label_i])[0][0]]
print label_cat

object


In [43]:
fieldmap = blob_info.fieldmap

In [209]:
pf = loadseg.SegmentationPrefetcher(ds, categories=['texture'], once=True, batch_size=128, 
                                    ahead=4, thread=False)

In [210]:
batch = pf.fetch_batch()

In [213]:
x = [rec for rec in batch if rec['texture'] != []]

In [216]:
23x[0]['texture'])

list

In [45]:
perc_label = np.zeros(len(idx))

In [46]:
i = 0
for batch in pf.batches():
    for rec in batch:
        sw, sh = [rec[k] for k in ['sw', 'sh']]
        perc_label[i] = np.sum(rec[label_cat] == label_i) / float(sw * sh)
        i += 1

In [48]:
alpha = float(1.-np.mean(perc_label)) # need float typecast
print alpha

0.878479830454


In [49]:
def run_epoch(activations, label_cat, label_i, fieldmap, thresh, loader, model, criterion, optimizer, epoch, train=True, 
              cuda=False, iou_threshold=0.5):
    if train:
        model.train()
        volatile=False
    else:
        model.eval()
        volatile=True
    
    batch_size = loader.batch_size
    N = activations.shape[0]

    losses = AverageMeter()
    iou_intersects = AverageMeter()
    iou_unions = AverageMeter()
    
    i = 0
    for batch in loader.batches():
        start = time.time()
        if (i+1)*batch_size < N:
            idx = range(i*batch_size, (i+1)*batch_size)
        else:
            idx = range(i*batch_size, N)
        i += 1
        up = [upsample.upsampleL(fieldmap, act, shape=(113,113), reduction=2) for act in activations[idx]]  
        input = torch.Tensor((up > thresh).astype(float))
        input_var = (Variable(input.cuda(), volatile=volatile) if cuda
                    else Variable(input, volatile=volatile))
        target = torch.Tensor([np.squeeze((rec[label_cat] == label_i).astype(float)) for rec in batch])
        target_var = (Variable(target.cuda(), volatile=volatile) if cuda
                     else Variable(target, volatile=volatile))
        output_var = model(input_var)
        loss = criterion(output_var, target_var)
        losses.update(loss.data[0], input.size(0))
        iou_intersects.update(iou_intersect(output_var, target_var, iou_threshold).data.cpu().numpy())
        iou_unions.update(iou_union(output_var, target_var, iou_threshold).data.cpu().numpy())
        
        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        iou = np.true_divide(iou_intersects.sum, iou_unions.sum)[0]

        if train:
            print('Epoch {0}[{1}/{2}]\t'
                  'Avg Loss {losses.avg:.4f}\t'
                  'Overall IOU {3}\t'
                  'Time {4}\t'.format(epoch, i, int(round(N/batch_size)), iou, time.time()-start, losses=losses))
        else:
            print('Test [{0}/{1}]\t'
                  'Avg Loss {losses.avg:.4f}\t'
                  'Overall IOU {2}\t'
                  'Time {3}\t'.format(i, int(round(N/batch_size)), iou, time.time()-start, losses=losses))

    if train:
        print('Epoch {0}\t'
              'Avg Loss {losses.avg:.4f}\t'
              'Overall IOU {1}\t'
              'Time {2}\t'.format(epoch, iou, time.time()-start, losses=losses))
    else:
        print('Test\t'
              'Avg Loss {losses.avg:.4f}\t'
              'Overall IOU {0}\t'
              'Time {1}\t'.format(iou, time.time()-start, losses=losses))

    return (losses.avg, iou)

In [50]:
criterion = lambda x,y: BCELoss2d(x,y,alpha)
layer = CustomLayer(blob_info.shape[1], upsample=False, act=True, positive=False)
layer.cuda()

CustomLayer (
  (activation): Sigmoid ()
)

In [51]:
starting_lr = 1e-5
momentum = 0.9
nesterov = False
lower_bound = None

optimizer = Custom_SGD(layer.parameters(), starting_lr, momentum=momentum,
                      l1_weight_decay=0, l2_weight_decay=0, nesterov=nesterov,
                      lower_bound=lower_bound)

In [52]:
for i in range(30):
    pf = loadseg.SegmentationPrefetcher(ds, categories=categories, indexes=idx, once=True, batch_size=48, 
                                    ahead=4, thread=False)
    (_, iou) = run_epoch(blobdata[idx], label_cat, label_i, fieldmap, thresh, pf, layer, criterion, optimizer, i+1, 
                         train=True, cuda=True, iou_threshold=0.5)

Epoch 1[1/197]	Avg Loss -1397.2006	Overall IOU 0.090954169631	Time 8.15757894516	
Epoch 1[2/197]	Avg Loss -1378.0912	Overall IOU 0.0817742720246	Time 8.21285200119	
Epoch 1[3/197]	Avg Loss -1341.0852	Overall IOU 0.0822424590588	Time 8.22417998314	
Epoch 1[4/197]	Avg Loss -1362.4483	Overall IOU 0.091764755547	Time 8.14122200012	
Epoch 1[5/197]	Avg Loss -1383.4397	Overall IOU 0.0899257212877	Time 8.17720293999	
Epoch 1[6/197]	Avg Loss -1385.8980	Overall IOU 0.0913786813617	Time 8.30944991112	
Epoch 1[7/197]	Avg Loss -1379.5078	Overall IOU 0.0934709236026	Time 8.32680606842	
Epoch 1[8/197]	Avg Loss -1353.6184	Overall IOU 0.0906980037689	Time 9.2346367836	
Epoch 1[9/197]	Avg Loss -1352.5752	Overall IOU 0.0950002893806	Time 8.41037893295	
Epoch 1[10/197]	Avg Loss -1347.1197	Overall IOU 0.0945718958974	Time 8.92816305161	
Epoch 1[11/197]	Avg Loss -1363.4478	Overall IOU 0.0981348156929	Time 8.12424492836	
Epoch 1[12/197]	Avg Loss -1358.9353	Overall IOU 0.0970431268215	Time 8.07023596764	
Epoc

Epoch 1[100/197]	Avg Loss -1357.4040	Overall IOU 0.141357704997	Time 8.15542507172	
Epoch 1[101/197]	Avg Loss -1355.9304	Overall IOU 0.141255378723	Time 8.23509311676	
Epoch 1[102/197]	Avg Loss -1356.9061	Overall IOU 0.141230657697	Time 8.25861620903	
Epoch 1[103/197]	Avg Loss -1357.4211	Overall IOU 0.141360342503	Time 8.24977993965	
Epoch 1[104/197]	Avg Loss -1357.1812	Overall IOU 0.141429305077	Time 8.65705680847	
Epoch 1[105/197]	Avg Loss -1356.5854	Overall IOU 0.141517877579	Time 8.37596297264	
Epoch 1[106/197]	Avg Loss -1356.0290	Overall IOU 0.141964584589	Time 9.43002295494	
Epoch 1[107/197]	Avg Loss -1355.6064	Overall IOU 0.14210344851	Time 8.34557080269	
Epoch 1[108/197]	Avg Loss -1356.0453	Overall IOU 0.142255082726	Time 8.10601520538	
Epoch 1[109/197]	Avg Loss -1354.7134	Overall IOU 0.142407000065	Time 8.21219301224	
Epoch 1[110/197]	Avg Loss -1357.4470	Overall IOU 0.143044412136	Time 8.09740996361	
Epoch 1[111/197]	Avg Loss -1357.9752	Overall IOU 0.143423989415	Time 8.295748

Epoch 1[198/197]	Avg Loss -1380.3163	Overall IOU 0.163852244616	Time 7.973747015	
Epoch 1	Avg Loss -1380.3163	Overall IOU 0.163852244616	Time 7.97384214401	
Epoch 2[1/197]	Avg Loss -1439.3044	Overall IOU 0.196214705706	Time 8.23317193985	
Epoch 2[2/197]	Avg Loss -1418.9155	Overall IOU 0.178265824914	Time 8.30884194374	
Epoch 2[3/197]	Avg Loss -1374.3008	Overall IOU 0.171232566237	Time 8.29010391235	
Epoch 2[4/197]	Avg Loss -1400.8173	Overall IOU 0.188434898853	Time 8.44455695152	
Epoch 2[5/197]	Avg Loss -1424.6591	Overall IOU 0.192083194852	Time 8.22908401489	
Epoch 2[6/197]	Avg Loss -1427.0508	Overall IOU 0.190543755889	Time 8.08816289902	
Epoch 2[7/197]	Avg Loss -1419.2682	Overall IOU 0.189567059278	Time 8.22137093544	
Epoch 2[8/197]	Avg Loss -1392.7402	Overall IOU 0.188194900751	Time 8.196600914	
Epoch 2[9/197]	Avg Loss -1392.7717	Overall IOU 0.191736415029	Time 8.20678806305	
Epoch 2[10/197]	Avg Loss -1386.6444	Overall IOU 0.188649564981	Time 8.13800692558	
Epoch 2[11/197]	Avg Loss

Epoch 2[99/197]	Avg Loss -1396.7098	Overall IOU 0.199204713106	Time 8.24038314819	
Epoch 2[100/197]	Avg Loss -1395.2717	Overall IOU 0.198634460568	Time 8.24918603897	
Epoch 2[101/197]	Avg Loss -1393.6709	Overall IOU 0.198261588812	Time 8.23187613487	
Epoch 2[102/197]	Avg Loss -1394.5164	Overall IOU 0.198113709688	Time 8.32574605942	
Epoch 2[103/197]	Avg Loss -1394.9620	Overall IOU 0.198046296835	Time 8.28783321381	
Epoch 2[104/197]	Avg Loss -1394.6189	Overall IOU 0.197940081358	Time 8.23382902145	
Epoch 2[105/197]	Avg Loss -1393.9304	Overall IOU 0.197930559516	Time 8.46462082863	
Epoch 2[106/197]	Avg Loss -1393.4425	Overall IOU 0.198253691196	Time 8.30036497116	
Epoch 2[107/197]	Avg Loss -1392.9180	Overall IOU 0.198319673538	Time 8.83255386353	
Epoch 2[108/197]	Avg Loss -1393.2732	Overall IOU 0.198182493448	Time 8.33843898773	
Epoch 2[109/197]	Avg Loss -1391.8234	Overall IOU 0.198082000017	Time 8.39949703217	
Epoch 2[110/197]	Avg Loss -1394.5839	Overall IOU 0.198319345713	Time 8.324950

KeyboardInterrupt: 

In [78]:
object_segs = [img['object'] for img in ds.image if len(img['object']) > 0]

In [60]:
pf = loadseg.SegmentationPrefetcher(ds, categories=[label_cat], split='val', indexes=idx, once=True, batch_size=48, 
                                ahead=4, thread=False)

In [63]:
len(pf.indexes)

2845

In [58]:
batch = pf.fetch_batch()

In [59]:
batch

[{'fn': u'/home/ruthfong/NetDissect/dataset/broden1_227/images/pascal/2009_004227.jpg',
  'i': 44,
  'object': array([[[12, 12, 12, ..., 12, 12, 12],
          [12, 12, 12, ..., 12, 12, 12],
          [12, 12, 12, ..., 12, 12, 12],
          ..., 
          [14, 14, 14, ..., 14, 14, 14],
          [14, 14, 14, ..., 14, 14, 14],
          [ 0,  0,  0, ...,  0,  0,  0]]], dtype=int16),
  'sh': 113,
  'sw': 113},
 {'fn': u'/home/ruthfong/NetDissect/dataset/broden1_227/images/pascal/2008_000204.jpg',
  'i': 51,
  'object': array([[[16, 16, 16, ..., 16, 16, 16],
          [16, 16, 16, ..., 16, 16, 16],
          [16, 16, 16, ..., 16, 16, 16],
          ..., 
          [52, 52, 52, ..., 52, 52, 52],
          [52, 52, 52, ..., 52, 52, 52],
          [52, 52, 52, ..., 52, 52, 52]]], dtype=int16),
  'sh': 113,
  'sw': 113},
 {'fn': u'/home/ruthfong/NetDissect/dataset/broden1_227/images/ade20k/ADE_train_00017934.jpg',
  'i': 56,
  'object': array([[[ 0,  0,  0, ...,  0,  0,  0],
          [ 0, 

In [82]:
dataset_dir = '/home/ruthfong/NetDissect/dataset/broden1_227/images'
seg_path = os.path.join(dataset_dir, object_segs[0][0])

In [71]:
from indexdata import load_image_to_label

In [83]:
weights = layer.weight.data.cpu().numpy()

In [84]:
weight_mmap = ed.open_mmap(blob=blob, part='label_i_%d_weights' % label_i, mode='w+', dtype=float, shape=weights.shape)

In [85]:
weight_mmap[:] = weights[:]
ed.finish_mmap(weight_mmap)

In [87]:
weight_mmap = ed.open_mmap(blob=blob, part='label_i_%d_weights' % label_i, mode='r')

In [90]:
np.array(weight_mmap)

array([ -3.68934881e+19,   9.73501265e-01,  -0.00000000e+00,
        -1.14430201e+00,   2.00000000e+00,  -1.10182035e+00,
        -3.68934881e+19,  -1.48505020e+00,   3.68934881e+19,
         1.52106559e+00,  -1.08420217e-19,  -1.26963007e+00,
        -1.08420217e-19,   1.46636283e+00,   1.08420217e-19,
        -1.21861231e+00,  -2.00000000e+00,  -1.37963653e+00,
         2.00000000e+00,   1.15191066e+00,   1.08420217e-19,
        -1.13857853e+00,   3.68934881e+19,   9.99566317e-01,
         1.08420217e-19,  -1.39639914e+00,  -0.00000000e+00,
        -1.31756771e+00,   0.00000000e+00,   1.21551323e+00,
         1.08420217e-19,  -1.00583649e+00,  -1.08420217e-19,
        -1.27421308e+00,  -0.00000000e+00,   1.59739411e+00,
         0.00000000e+00,  -1.36832309e+00,   0.00000000e+00,
        -1.28661191e+00,  -3.68934881e+19,   1.38106072e+00,
         2.00000000e+00,  -1.22277212e+00,  -3.68934881e+19,
         1.37986910e+00,   0.00000000e+00,  -1.42640877e+00,
         1.08420217e-19,

In [4]:
from torchvision import models

In [5]:
net = models.alexnet(pretrained=True)

In [6]:
net

AlexNet (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU (inplace)
    (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU (inplace)
    (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU (inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU (inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  )
  (classifier): Sequential (
    (0): Dropout (p = 0.5)
    (1): Linear (9216 -> 4096)
    (2): ReLU (inplace)
    (3): Dropout (p = 0.5)
    (4): Linear (4096 -> 4096)
    (5): ReLU (inplace)
    (6): Linear (4096 -> 1000)
  )
)

In [137]:
list(net.modules())

[AlexNet (
   (features): Sequential (
     (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
     (1): ReLU (inplace)
     (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
     (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
     (4): ReLU (inplace)
     (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
     (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (7): ReLU (inplace)
     (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (9): ReLU (inplace)
     (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (11): ReLU (inplace)
     (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
   )
   (classifier): Sequential (
     (0): Dropout (p = 0.5)
     (1): Linear (9216 -> 4096)
     (2): ReLU (inplace)
     (3): Dropout (p = 0.5)
     (4): Linear (4096 -> 4096)
     (5): ReLU (inplace)
     (6): Linear (4096 -> 1000)
   )
 ), S

In [136]:
for i, m in enumerate(net.children()):
    print i, m

0 Sequential (
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU (inplace)
  (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU (inplace)
  (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU (inplace)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU (inplace)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU (inplace)
  (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
)
1 Sequential (
  (0): Dropout (p = 0.5)
  (1): Linear (9216 -> 4096)
  (2): ReLU (inplace)
  (3): Dropout (p = 0.5)
  (4): Linear (4096 -> 4096)
  (5): ReLU (inplace)
  (6): Linear (4096 -> 1000)
)


In [98]:
net.features[0]

Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))

In [138]:
blob = 'features.2'

In [113]:
module_name, module_index = blob.split('.')
module_index = int(module_index)

In [143]:
def get_pytorch_module(net, blob):
    modules = blob.split('.')
    if len(modules) == 1:
        return net._modules(get(blob))
    else:
        curr_m = net
        for m in modules:
            curr_m = curr_m._modules.get(m)
        return curr_m

In [144]:
print get_module(net, 'features.2')

MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))


In [114]:
net._modules[module_name][module_index]

MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))

In [118]:
print net._modules.get('features')

Sequential (
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU (inplace)
  (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU (inplace)
  (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU (inplace)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU (inplace)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU (inplace)
  (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
)


In [119]:
z = np.zeros(10, dtype=bool)

In [120]:
z

array([False, False, False, False, False, False, False, False, False, False], dtype=bool)

In [145]:
net = models.__dict__['inception_v3'](pretrained=True)

In [146]:
net

Inception3 (
  (Conv2d_1a_3x3): BasicConv2d (
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True)
  )
  (Conv2d_2a_3x3): BasicConv2d (
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True)
  )
  (Conv2d_2b_3x3): BasicConv2d (
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True)
  )
  (Conv2d_3b_1x1): BasicConv2d (
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True)
  )
  (Conv2d_4a_3x3): BasicConv2d (
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
  )
  (Mixed_5b): InceptionA (
    (branch1x1): BasicConv2d (
      (conv): Conv2d(192, 64, kernel_

In [147]:
print get_module(net, 'Mixed_5c.branch_pool.conv')

Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
